# Experimenting with MAgent2 Environment

In [ ]:
from magent2.environments import battle_v4
import numpy as np


In [ ]:


env = battle_v4.env(map_size=20, render_mode='human', max_cycles=100000)
env.reset()
for agent in env.agent_iter():
    observation, reward, termination, truncation, info = env.last()
    if  termination or truncation:
        env.step(None)
    else:
        action = env.action_space(agent).sample()
        env.step(action)

    # If a team all terminated, break

env.close()

In [ ]:
env = battle_v4.env(map_size=45, render_mode="rgb_array", max_cycles=100000)
env.observation_spaces

from functools import reduce
env.observation_spaces['red_0'].shape
reduce((lambda x, y: x * y), env.observation_spaces['red_0'].shape)
# env.observation_space('red_0').sample().flatten().shape

In [ ]:
env.reset()
env.action_spaces['red_0'].n
type (env)


## Parallel API

In [ ]:
from magent2.environments import battle_v4
env = battle_v4.parallel_env(map_size=20, render_mode='human', max_cycles=100000)
env.reset()['red_0']
# env.reset()

# terminated_agents = set()
# while env.agents:
#     actions = {agent: env.action_space(agent).sample() for agent in env.agents}
#     for agent in terminated_agents:
#         actions[agent] = None
#
#     observations, rewards, terminations, truncations, infos = env.step(actions)
#
#     # Terminate agents that are done
#     terminated_agents = {agent for agent, done in terminations.items() if done}
#     truncated_agents = {agent for agent, done in truncations.items() if done}
#     terminated_agents.update(truncated_agents)
#
# env.close()

In [3]:
from magent2.environments import tiger_deer_v4
env = tiger_deer_v4.parallel_env(map_size=20, render_mode='human', max_cycles=1000, tiger_step_recover=0)
env.reset()
# env.reset()
cnt_cycle = 0

terminated_agents = set()
acc_rewards = {agent: 0 for agent in env.agents}
action_stats = {agent: {action: 0 for action in range(0, 14)} for agent in env.agents}
while env.agents:
    actions = {agent: env.action_space(agent).sample() for agent in env.agents}
    for agent in terminated_agents:
        actions[agent] = None

    observations, rewards, terminations, truncations, infos = env.step(actions)

    # Terminate agents that are done
    terminated_agents = {agent for agent, done in terminations.items() if done}
    truncated_agents = {agent for agent, done in truncations.items() if done}
    terminated_agents.update(truncated_agents)

    for agent in env.agents:
        acc_rewards[agent] += rewards[agent]
        action_stats[agent][actions[agent]] += 1

    cnt_cycle += 1
    if cnt_cycle % 100 == 0:
        with open('../data/cycle.txt', 'a') as f:
            f.write(f'{cnt_cycle}, {action_stats}\n')
            f.write(f'{cnt_cycle}, {acc_rewards}\n')

env.close()